<a href="https://colab.research.google.com/github/KlimLe/ML4B-Stock-Prediction/blob/main/BasicModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas numpy spacy yfinance scikit-learn transformers tensorflow nltk ta textblob

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29411 sha256=3343e4a1620b76252bae185ade5fae9fa7b114fa8fc7f2915813694131aaf54d
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [ ]:
import pandas as pd
import numpy as np
import spacy
import yfinance as yf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.impute import KNNImputer
from transformers import RobertaTokenizer, TFRobertaModel
import tensorflow as tf
import nltk
from ta import add_all_ta_features
from textblob import TextBlob
import re

# Load new financial news dataset
news_data = pd.read_csv('/content/final_dataset.csv')  # Replace with your dataset path
news_data['Date'] = pd.to_datetime(news_data['Date'])
news_data.rename(columns={'News Article': 'News_Article', 'Date': 'Date'}, inplace=True)

# Download NLTK data
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize Spacy model and NLTK components
nlp = spacy.load("en_core_web_sm")
stop_words = set(nltk.corpus.stopwords.words('english'))
lemmatizer = nltk.stem.WordNetLemmatizer()

# List of companies to focus on
companies_to_focus = {
    'AMZN': 'Amazon',
    'GOOGL': 'Google',
    'AAPL': 'Apple'
}

# Function to preprocess text
def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text, re.I | re.A)
    text = text.lower()
    text = text.strip()
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    processed_text = ' '.join(tokens)
    return processed_text

# Preprocess news articles
news_data['Processed_Article'] = news_data['News_Article'].apply(preprocess_text)

# Perform Sentiment Analysis
def get_sentiment(text):
    return TextBlob(text).sentiment.polarity

news_data["Sentiment"] = news_data["Processed_Article"].apply(get_sentiment)

# Initialize BERT tokenizer and model (You can also use RoBERTa or other advanced models)
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
bert_model = TFRobertaModel.from_pretrained('roberta-base')

def get_bert_embeddings(texts, tokenizer, model):
    inputs = tokenizer(texts, return_tensors="tf", padding=True, truncation=True, max_length=128)
    outputs = model(inputs)
    return outputs.last_hidden_state[:, 0, :].numpy()  # Use the [CLS] token's embedding

# Calculate BERT embeddings for all news
news_data["BERT_Embedding"] = news_data["Processed_Article"].apply(lambda x: get_bert_embeddings([x], tokenizer, bert_model)[0])

# Function to fetch stock prices and fundamental data for each company
def fetch_stock_prices(ticker, start_date, end_date):
    try:
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        if stock_data.shape[0] > 14:  # Ensure there are at least 15 rows of data
            stock_data = add_all_ta_features(stock_data, open="Open", high="High", low="Low", close="Close", volume="Volume")
            # Handle missing technical indicators
            imputer = KNNImputer(n_neighbors=5)
            stock_data.iloc[:, :] = imputer.fit_transform(stock_data)
        else:
            print(f"Not enough data for {ticker}")
            return pd.DataFrame()

        # Filter out rows with missing stock prices
        stock_data.dropna(subset=['Close'], inplace=True)

        # Reset index to get the date column back after filtering
        stock_data.reset_index(inplace=True)

        return stock_data
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return pd.DataFrame()

def fetch_fundamental_data(ticker):
    stock = yf.Ticker(ticker)
    fundamentals = stock.info
    return {
        "PE_Ratio": fundamentals.get("trailingPE", np.nan),
        "EPS": fundamentals.get("trailingEps", np.nan),
        "Revenue": fundamentals.get("totalRevenue", np.nan),
        "Market_Cap": fundamentals.get("marketCap", np.nan)
    }

# Correct date format and optionally extend the date range
from_date = "2021-01-01"
to_date = "2021-12-31"  # Extended date range

# Define look-back window
look_back = 5

# Function to prepare data for each company
def prepare_company_data(ticker, company, from_date, to_date):
    print(f"Fetching data for {company} ({ticker})")
    stock_data = fetch_stock_prices(ticker, from_date, to_date)
    if stock_data.empty:
        print(f"No stock data found for {company} ({ticker})")
        return None
    fundamental_data = fetch_fundamental_data(ticker)

    # Filter news for the company or its ticker symbol
    company_news = news_data[news_data['News_Article'].str.contains(company, case=False) | news_data['News_Article'].str.contains(ticker, case=False)]

    # Aggregate all news by day
    all_news_agg = news_data.groupby('Date').agg({
        'BERT_Embedding': lambda x: np.mean(np.vstack(x), axis=0),
        'Sentiment': 'mean'
    }).reset_index()

    # Handle missing dates for all news
    all_dates = pd.date_range(start=from_date, end=to_date, freq='D')
    all_news_agg = all_news_agg.set_index('Date').reindex(all_dates).reset_index()
    all_news_agg.rename(columns={'index': 'Date'}, inplace=True)

    # Insert neutral values for missing dates
    all_news_agg['BERT_Embedding'] = all_news_agg['BERT_Embedding'].apply(lambda x: x if isinstance(x, np.ndarray) else np.zeros(bert_model.config.hidden_size))
    all_news_agg['Sentiment'] = all_news_agg['Sentiment'].fillna(0.0)

    # Aggregate company-specific news by day
    if not company_news.empty:
        company_news_agg = company_news.groupby('Date').agg({
            'BERT_Embedding': lambda x: np.mean(np.vstack(x), axis=0),
            'Sentiment': 'mean'
        }).reset_index()

        # Handle missing dates for company-specific news
        company_news_agg = company_news_agg.set_index('Date').reindex(all_dates).reset_index()
        company_news_agg.rename(columns={'index': 'Date'}, inplace=True)

        # Insert neutral values for missing dates
        company_news_agg['BERT_Embedding'] = company_news_agg['BERT_Embedding'].apply(lambda x: x if isinstance(x, np.ndarray) else np.zeros(bert_model.config.hidden_size))
        company_news_agg['Sentiment'] = company_news_agg['Sentiment'].fillna(0.0)
    else:
        # Create empty DataFrame with the same structure
        company_news_agg = pd.DataFrame({
            'Date': all_dates,
            'BERT_Embedding': [np.zeros(bert_model.config.hidden_size)] * len(all_dates),
            'Sentiment': [0.0] * len(all_dates)
        })

    # Ensure the columns have correct suffixes
    company_news_agg.rename(columns={'BERT_Embedding': 'BERT_Embedding_company', 'Sentiment': 'Sentiment_company'}, inplace=True)
    all_news_agg.rename(columns={'BERT_Embedding': 'BERT_Embedding_all', 'Sentiment': 'Sentiment_all'}, inplace=True)

    # Merge stock data with aggregated news data
    data = pd.merge(stock_data, company_news_agg, on="Date", how="left")
    data = pd.merge(data, all_news_agg, on="Date", how="left")

    # Add fundamental data (same value for all rows as an example)
    for key, value in fundamental_data.items():
        data[key] = value

    data["Company_Name"] = company

    # Add future price column
    data["Future_Price"] = data["Close"].shift(-1)  # Shift price for prediction

    # Drop rows where the future price is missing (typically the last row)
    data.dropna(subset=['Future_Price'], inplace=True)

    # Impute missing values in technical indicators and fundamentals
    technical_indicator_columns = data.filter(like='ta_').columns
    for column in technical_indicator_columns:
        data[column].fillna(method='ffill', inplace=True)
        data[column].fillna(method='bfill', inplace=True)

    fundamental_columns = ["PE_Ratio", "EPS", "Revenue", "Market_Cap"]
    for column in fundamental_columns:
        data[column].fillna(method='ffill', inplace=True)
        data[column].fillna(method='bfill', inplace=True)

    return data

# Prepare data for each company
all_company_data = {ticker: prepare_company_data(ticker, company, from_date, to_date) for ticker, company in companies_to_focus.items()}

# Check for and remove any None entries
all_company_data = {ticker: data for ticker, data in all_company_data.items() if data is not None}

if not all_company_data:
    raise ValueError("No data available for any company in the specified date range.")

# Create sequences for each company
def create_sequences(data, look_back):
    sequences = []
    targets = []
    for i in range(len(data) - look_back):
        sequence = {
            "news_embeddings_company": np.stack(data["BERT_Embedding_company"].values[i:i+look_back]),
            "news_embeddings_all": np.stack(data["BERT_Embedding_all"].values[i:i+look_back]),
            "price": data["Close"].values[i:i+look_back].reshape(-1, 1),
            "sentiment_company": data["Sentiment_company"].values[i:i+look_back].reshape(-1, 1),
            "sentiment_all": data["Sentiment_all"].values[i:i+look_back].reshape(-1, 1),
            "technical_indicators": data.filter(like='ta_').values[i:i+look_back],
            "fundamentals": data[["PE_Ratio", "EPS", "Revenue", "Market_Cap"]].values[i:i+look_back]
        }
        sequences.append(sequence)
        targets.append(data["Future_Price"].values[i + look_back])  # Correctly assign the future price as target
    return sequences, np.array(targets)

company_sequences = {ticker: create_sequences(data, look_back) for ticker, data in all_company_data.items()}

# Ensure consistency of lengths
min_length = min(len(sequences) for sequences, _ in company_sequences.values())
company_sequences = {ticker: (sequences[:min_length], targets[:min_length]) for ticker, (sequences, targets) in company_sequences.items()}

# Convert sequences to arrays for model input
def convert_sequences(sequences):
    news_embeddings_company = np.array([seq["news_embeddings_company"] for seq in sequences])
    news_embeddings_all = np.array([seq["news_embeddings_all"] for seq in sequences])
    price = np.array([seq["price"] for seq in sequences])
    sentiment_company = np.array([seq["sentiment_company"] for seq in sequences])
    sentiment_all = np.array([seq["sentiment_all"] for seq in sequences])
    technical_indicators = np.array([seq["technical_indicators"] for seq in sequences])
    fundamentals = np.array([seq["fundamentals"] for seq in sequences])
    return news_embeddings_company, news_embeddings_all, price, sentiment_company, sentiment_all, technical_indicators, fundamentals

company_features = {ticker: (convert_sequences(sequences), targets) for ticker, (sequences, targets) in company_sequences.items()}

# Validate lengths of the features
for key, (value, targets) in company_features.items():
    print(f"{key} lengths: {[len(x) for x in value]}, targets length: {len(targets)}")
# Combine all features into a single array
def combine_features(features):
    combined = np.concatenate([features[0],
                               features[1],
                               features[2],
                               features[3],
                               features[4],
                               features[5],
                               features[6]], axis=-1)
    return combined

combined_features = {ticker: combine_features(features) for ticker, (features, _) in company_features.items()}
combined_features_array = np.concatenate(list(combined_features.values()), axis=0)

# Concatenate all targets into a single array along the correct axis
targets_array = np.concatenate([targets.reshape(-1, 1) for _, targets in company_features.values()], axis=0)

# Ensure the shape of targets matches the expected dimensions
targets_array = targets_array.reshape(-1, len(companies_to_focus))

# Convert targets to a DataFrame for multi-output regression
targets_df = pd.DataFrame(targets_array, columns=companies_to_focus.keys())

# Scale features
scaler = StandardScaler()
combined_features_array_scaled = scaler.fit_transform(combined_features_array.reshape(-1, combined_features_array.shape[-1]))
combined_features_array_scaled = combined_features_array_scaled.reshape(combined_features_array.shape)

# Scale the targets (future prices) individually for each company
target_scalers = {ticker: StandardScaler() for ticker in companies_to_focus.keys()}
targets_array_scaled = np.zeros_like(targets_array)

for i, ticker in enumerate(companies_to_focus.keys()):
    targets_array_scaled[:, i] = target_scalers[ticker].fit_transform(targets_array[:, i].reshape(-1, 1)).flatten()

# Convert targets to a DataFrame for multi-output regression
targets_df_scaled = pd.DataFrame(targets_array_scaled, columns=companies_to_focus.keys())

# Ensure the number of samples is the same
if combined_features_array.shape[0] != targets_df_scaled.shape[0]:
    min_samples = min(combined_features_array.shape[0], targets_df_scaled.shape[0])
    combined_features_array = combined_features_array[:min_samples]
    targets_df_scaled = targets_df_scaled.iloc[:min_samples]

# Prepare your data
tscv = TimeSeriesSplit(n_splits=5)
for train_index, val_index in tscv.split(combined_features_array):
    X_train, X_val = combined_features_array[train_index], combined_features_array[val_index]
    y_train, y_val = targets_df_scaled.values[train_index], targets_df_scaled.values[val_index]

# Define the model
def build_model(look_back, combined_dim, num_companies, num_heads=12, ff_dim=128, dropout_rate=0.5):
    combined_input = tf.keras.layers.Input(shape=(look_back, combined_dim), name='combined_input')

    # Register the custom layer for deserialization
    @tf.keras.utils.register_keras_serializable()
    # Transformer block
    class TransformerBlock(tf.keras.layers.Layer):
        def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
            super(TransformerBlock, self).__init__()
            self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
            self.ffn = tf.keras.Sequential([
                tf.keras.layers.Dense(ff_dim, activation="relu"),
                tf.keras.layers.Dense(embed_dim),
            ])
            self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
            self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
            self.dropout1 = tf.keras.layers.Dropout(rate)
            self.dropout2 = tf.keras.layers.Dropout(rate)

        def call(self, inputs, training):
            attn_output = self.att(inputs, inputs)
            attn_output = self.dropout1(attn_output, training=training)
            out1 = self.layernorm1(inputs + attn_output)
            ffn_output = self.ffn(out1)
            ffn_output = self.dropout2(ffn_output, training=training)
            return self.layernorm2(out1 + ffn_output)

    transformer_block = TransformerBlock(combined_dim, num_heads, ff_dim, rate=dropout_rate)
    x = transformer_block(combined_input)

    # Global average pooling
    x = tf.keras.layers.GlobalAveragePooling1D()(x)

    # Dense layer with Batch Normalization and Dropout
    x = tf.keras.layers.Dense(64, activation="relu")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)

    # Output layers for each company
    outputs = {ticker: tf.keras.layers.Dense(1, activation='linear', name=f'output_{ticker}')(x) for ticker in companies_to_focus.keys()}

    # Create model
    model = tf.keras.models.Model(inputs=combined_input, outputs=outputs)

    # Compile model with a dictionary of losses
    losses = {ticker: 'mse' for ticker in companies_to_focus.keys()}
    model.compile(loss=losses, optimizer=tf.keras.optimizers.Adam())

    return model

look_back = 5  # Define the look_back as per your data
combined_dim = combined_features_array.shape[-1]  # Combined dimension

model = build_model(look_back, combined_dim, len(companies_to_focus), 12, 128, 0.5)

# Define the number of epochs
epochs = 50

# Set batch size
batch_size = 32

# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size, callbacks=[early_stopping])

# Make predictions on validation data
predicted_prices_scaled = model.predict(X_val)

# Inverse transform the predictions to get the original scale
predicted_prices = {ticker: target_scalers[ticker].inverse_transform(predictions) for ticker, predictions in predicted_prices_scaled.items()}

# Display the predicted prices in the original scale
print(predicted_prices)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Fetching data for Google (GOOGL)


Fetching data for Apple (AAPL)


[*********************100%%**********************]  1 of 1 completed


AMZN lengths: [245, 245, 245, 245, 245, 245, 245], targets length: 245
GOOGL lengths: [245, 245, 245, 245, 245, 245, 245], targets length: 245
AAPL lengths: [245, 245, 245, 245, 245, 245, 245], targets length: 245
Epoch 1/50
7/7 [==============================] - 56s 5s/step - loss: 11.1212 - output_AAPL_loss: 3.3236 - output_AMZN_loss: 3.7104 - output_GOOGL_loss: 4.0871 - val_loss: 41.4114 - val_output_AAPL_loss: 14.9877 - val_output_AMZN_loss: 24.1647 - val_output_GOOGL_loss: 2.2590
Epoch 2/50
7/7 [==============================] - 33s 5s/step - loss: 9.4883 - output_AAPL_loss: 3.0376 - output_AMZN_loss: 3.0807 - output_GOOGL_loss: 3.3700 - val_loss: 16.4944 - val_output_AAPL_loss: 3.8188 - val_output_AMZN_loss: 10.0812 - val_output_GOOGL_loss: 2.5943
Epoch 3/50
7/7 [==============================] - 37s 5s/step - loss: 8.3521 - output_AAPL_loss: 2.3178 - output_AMZN_loss: 2.8447 - output_GOOGL_loss: 3.1896 - val_loss: 3.3948 - val_output_AAPL_loss: 0.7234 - val_output_AMZN_loss: 2.2

In [ ]:
model.save('trained_model.h5')

/usr/local/lib/python3.10/dist-packages/tf_keras/src/engine/training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import yfinance as yf
import tensorflow as tf
from datetime import datetime, timedelta
from transformers import RobertaTokenizer, TFRobertaModel
import plotly.graph_objs as go
from textblob import TextBlob
import re
from sklearn.preprocessing import StandardScaler
import gdown

# Define the company tickers and names
companies_to_focus = {
    'AMZN': 'Amazon',
    'GOOGL': 'Google',
    'AAPL': 'Apple'
}

# Initialize tokenizer and BERT model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
bert_model = TFRobertaModel.from_pretrained('roberta-base')

# Define lookback window
look_back = 5

# Register the custom layer for deserialization
@tf.keras.utils.register_keras_serializable()
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1, **kwargs):
        super(TransformerBlock, self).__init__(**kwargs)
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            tf.keras.layers.Dense(ff_dim, activation="relu"),
            tf.keras.layers.Dense(embed_dim),
        ])
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

# Load the trained model with the custom layer
custom_objects = {'TransformerBlock': TransformerBlock}
model = tf.keras.models.load_model('/content/trained_model.h5', custom_objects=custom_objects)

# Function to preprocess text for BERT embeddings
def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text, re.I | re.A)
    text = text.lower().strip()
    tokens = text.split()
    return ' '.join(tokens)

# Function to get BERT embeddings
def get_bert_embeddings(texts, tokenizer, model):
    inputs = tokenizer(texts, return_tensors="tf", padding=True, truncation=True, max_length=128)
    outputs = model(inputs)
    return outputs.last_hidden_state[:, 0, :].numpy()  # Use the [CLS] token's embedding

# Function to predict future prices
def predict_prices(news_headlines, look_back_window, bert_dim, combined_dim, scaler, target_scalers):
    processed_articles = [preprocess_text(article) for article in news_headlines]
    bert_embeddings = [get_bert_embeddings([article], tokenizer, bert_model)[0] for article in processed_articles]

    # Ensure the embeddings have the correct shape
    bert_embeddings = bert_embeddings[-look_back_window:]
    if len(bert_embeddings) < look_back_window:
        # Pad the embeddings if there are not enough look-back days
        padding = [np.zeros((bert_dim,)) for _ in range(look_back_window - len(bert_embeddings))]
        bert_embeddings = padding + bert_embeddings

    if combined_dim > bert_dim:
        # Combine with dummy data to match the expected combined dimension
        dummy_data = np.zeros((look_back_window, combined_dim - bert_dim))
        combined_features = np.concatenate([bert_embeddings, dummy_data], axis=-1)
    else:
        combined_features = np.array(bert_embeddings)

    # Reshape for model input
    combined_features = np.array(combined_features).reshape(1, look_back_window, -1)

    # Scale the combined features
    combined_features_scaled = scaler.transform(combined_features.reshape(-1, combined_features.shape[-1]))
    combined_features_scaled = combined_features_scaled.reshape(combined_features.shape)

    # Predict using the loaded model
    predictions_scaled = model.predict(combined_features_scaled)

    # Inverse transform the predictions to get the original scale
    predictions = {ticker: target_scalers[ticker].inverse_transform(predictions_scaled[ticker]) for ticker in companies_to_focus.keys()}
    return predictions

# Function to perform sentiment analysis
def get_sentiment(text):
    return TextBlob(text).sentiment.polarity

# Function to fetch fundamental data for a company
def fetch_fundamental_data(ticker):
    stock = yf.Ticker(ticker)
    fundamentals = stock.info
    return {
        "PE_Ratio": fundamentals.get("trailingPE", np.nan),
        "EPS": fundamentals.get("trailingEps", np.nan),
        "Revenue": fundamentals.get("totalRevenue", np.nan),
        "Market_Cap": fundamentals.get("marketCap", np.nan)
    }

# Load the dataset
news_data = pd.read_csv('/content/final_dataset_without_last_column.csv')
news_data['Date'] = pd.to_datetime(news_data['Date'])
news_data['Processed_Article'] = news_data['News_Article'].apply(preprocess_text)
news_data['Sentiment'] = news_data['Processed_Article'].apply(get_sentiment)

# Define dimensions
bert_dim = bert_model.config.hidden_size  # typically 768 for BERT models
combined_dim = 1543  # Update this to the correct combined dimension

# Initialize scalers
scaler = StandardScaler()
target_scalers = {ticker: StandardScaler() for ticker in companies_to_focus.keys()}

# Simulate fitting scalers with initial data
def fit_scalers():
    combined_features_list = []
    targets_list = []

    for ticker in companies_to_focus.keys():
        # Simulate fetching stock data
        stock_data = yf.download(ticker, start='2021-01-01', end='2021-12-31')
        stock_data.reset_index(inplace=True)

        # Fetch moving averages
        ma50 = stock_data['Close'].rolling(window=50).mean()
        ma200 = stock_data['Close'].rolling(window=200).mean()

        stock_data['MA50'] = ma50
        stock_data['MA200'] = ma200

        # Generate dummy combined features matching the expected combined dimension
        num_samples = len(stock_data)
        dummy_bert_features = np.zeros((num_samples, 768))  # Example BERT feature size
        dummy_other_features = np.zeros((num_samples, combined_dim - 768))
        combined_features = np.hstack([dummy_bert_features, dummy_other_features])

        combined_features_list.append(combined_features)
        targets_list.append(stock_data['Close'].values)

    combined_features_array = np.concatenate(combined_features_list, axis=0)
    targets_array = np.concatenate(targets_list, axis=0).reshape(-1, len(companies_to_focus))

    scaler.fit(combined_features_array)
    for i, ticker in enumerate(companies_to_focus.keys()):
        target_scalers[ticker].fit(targets_array[:, i].reshape(-1, 1))

fit_scalers()

# Streamlit App Layout
st.title("Stock Price Prediction App")

# Sidebar Description
st.sidebar.title("About the App")
st.sidebar.markdown("""
This application predicts the stock prices of major companies using news headlines and sentiment analysis.
We utilize BERT embeddings, technical indicators, and fundamental data for robust predictions.
""")

st.sidebar.title("Model Description")
st.sidebar.markdown("""
Our model leverages a transformer-based architecture with BERT embeddings to capture the semantic meaning of news articles.
We incorporate technical indicators, such as moving averages, and fundamental data to improve the prediction accuracy.
""")

# Fetch data
today = datetime.today().strftime('%Y-%m-%d')
start_date = (datetime.today() - timedelta(days=365)).strftime('%Y-%m-%d')
end_date = today

# Get today's news headlines
todays_news = news_data[news_data['Date'] == today].head(6)  # Display at most 6 headlines

# Get stock data and predictions
stock_data_dict = {}
fundamental_data_dict = {}
for ticker in companies_to_focus:
    stock_data = yf.download(ticker, start=start_date, end=end_date)

    # Ensure the Date column is present
    stock_data.reset_index(inplace=True)

    # Fetch moving averages
    ma50 = stock_data['Close'].rolling(window=50).mean()
    ma200 = stock_data['Close'].rolling(window=200).mean()

    stock_data['MA50'] = ma50
    stock_data['MA200'] = ma200

    stock_data_dict[ticker] = stock_data
    fundamental_data_dict[ticker] = fetch_fundamental_data(ticker)

# Call predict_prices once
news_headlines = todays_news['Processed_Article'].tolist()
predictions = predict_prices(news_headlines, look_back, bert_dim, combined_dim, scaler, target_scalers)
predictions_dict = {ticker: predictions[ticker] for ticker in companies_to_focus}

# Display predicted prices for tomorrow
st.subheader("Predicted Prices for Tomorrow")
for ticker, company in companies_to_focus.items():
    today_price = stock_data_dict[ticker]['Close'].values[-1]
    predicted_price = predictions_dict[ticker][0][0]
    arrow = "⬆️" if predicted_price > today_price else "⬇️"
    color = "green" if predicted_price > today_price else "red"
    st.markdown(f"**{company} ({ticker}):** {predicted_price:.2f} {arrow}", unsafe_allow_html=True)

# Display news headlines with sentiment in a table
st.subheader("Latest News")
news_table = todays_news[['News_Article', 'Sentiment']].copy()
news_table.columns = ['News Article', 'Sentiment']
news_table['Sentiment'] = news_table['Sentiment'].apply(lambda x: f"<span style='color:{'green' if x > 0 else 'red'}'>{x:.2f}</span>")
st.write(news_table.to_html(escape=False, index=False), unsafe_allow_html=True)

# Manual prediction input
st.subheader("Manual Prediction Input")
manual_news_headlines = st.text_area("Enter News Headlines", "").split('\n')

if st.button("Predict Manually"):
    if manual_news_headlines:
        manual_predictions = predict_prices(manual_news_headlines, look_back, bert_dim, combined_dim, scaler, target_scalers)
        for ticker, company in companies_to_focus.items():
            manual_prediction = manual_predictions[ticker][0][0]
            today_price = stock_data_dict[ticker]['Close'].values[-1]
            arrow = "⬆️" if manual_prediction > today_price else "⬇️"
            st.write(f"Predicted price for {company} ({ticker}): {manual_prediction:.2f} {arrow}")

# Display stock price charts with actual, predicted prices, and technical indicators
for ticker, company in companies_to_focus.items():
    stock_data = stock_data_dict[ticker]
    fig = go.Figure()

    # Add actual stock price trace
    fig.add_trace(go.Scatter(x=stock_data['Date'], y=stock_data['Close'], mode='lines', name='Actual Close'))

    # Add predicted price trace
    predicted_price = predictions_dict[ticker][0][0]
    predicted_date = stock_data['Date'].iloc[-1] + timedelta(days=1)
    fig.add_trace(go.Scatter(x=[predicted_date], y=[predicted_price], mode='markers', name='Predicted Close', marker=dict(color='red', size=10)))

    # Add moving average traces
    fig.add_trace(go.Scatter(x=stock_data['Date'], y=stock_data['MA50'], mode='lines', name='MA50'))
    fig.add_trace(go.Scatter(x=stock_data['Date'], y=stock_data['MA200'], mode='lines', name='MA200'))

    # Customize the layout
    fig.update_layout(
        title=f'{company} ({ticker}) Stock Prices',
        xaxis_title='Date',
        yaxis_title='Price',
        showlegend=True
    )

    # Display the chart
    st.plotly_chart(fig)

    # Display fundamental data
    st.subheader(f"{company} ({ticker}) Fundamentals")
    fundamentals = fundamental_data_dict[ticker]
    st.markdown(f"""
    - **PE Ratio**: {fundamentals['PE_Ratio']}
    - **EPS**: {fundamentals['EPS']}
    - **Revenue**: {fundamentals['Revenue']}
    - **Market Cap**: {fundamentals['Market_Cap']}
    """)

# "See More" Section
st.subheader("See More")
st.markdown("""
We also trained a model that uses Topic Modelling, TF-IDF, and Named Entity Recognition (NER) as features.
For more details, check out our [GitHub Repository](https://github.com/your-repo-link).
""")

# End of the Streamlit app

Overwriting app.py


In [ ]:
!pip install streamlit pyngrok

In [ ]:
# Download the model from Google Drive
url = 'https://drive.google.com/uc?id=1MWn669IPHXC8gzyJk1WY_3BQIldVVKAM'  # Replace with your file ID
output = 'trained_model.h5'
gdown.download(url, output, quiet=False)

In [ ]:
from pyngrok import ngrok

# Initialize ngrok
ngrok.set_auth_token('2h6sfydzZ90UINBPKt8DX3rmu1h_6jzyuJACPKAhjFUh64RAx')  # Get your auth token from ngrok

# Kill any previous tunnels if open
ngrok.kill()

# Start a new ngrok tunnel
public_url = ngrok.connect(8501)

# Display the public URL
print(f'Streamlit App is available at: {public_url}')

# Run the Streamlit app
!streamlit run app.py &>/dev/null&

Streamlit App is available at: NgrokTunnel: "https://2c94-34-75-98-191.ngrok-free.app" -> "http://localhost:8501"
